# Homework 4: Practical analysis with BioPython

For the homework, you are going to extend the code from real biological analysis of our FASTQ files in lectures 8 and 9.

As described in the Jupyter notebook for that lecture, the FASTQ reads can originate from **either** HA or NA, and that will be distinguished by the most 3' end of the read.
But in our example, we did not distinguish among reads matching to HA and NA, as we didn't even look far enough into the read to tell the identity.

For the homework, your goal is to write code that extends the material from lectures 8 and 9 to also distinguish between HA and NA.

Please include code to address each of the following questions. Please include code comments to explain what your code is attempting to accomplish. Don't forget to include references to the sources you used to obtain your answer, including your classmates (if you are working in groups).  

## A real biological analysis: parsing barcodes
The reads that we just read as `seqreads_str` come from a real sequencing run of influenza virus HA and NA genes.

The sequences are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N]-3'
    
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N]-3'
    
The end of NA is:

    ...CACGATAGATAAATAATAGTGCACCAT
    
The end of HA is:

    ...CCGGATTTGCATATAATGATGCACCAT
    
The sequencing run reads from the reverse end of the molecules, so the first thing in the sequencing reads is the barcode followed by the constant sequence and the end of HA or NA.

1. How many reads map to HA, and how many reads map to NA?

In [6]:
#taken from lecture09

import re #imports regular expression package onto jpynb

import Bio.SeqIO #imports Bio python package onto jpynb


In [5]:
#taken from lecture09

seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', format='fastq')) #parses through barcodes_R1.fastq file and stores every entry in a list called seqreads
seqreads_str = [str(s.seq) for s in seqreads] #iterates through all entries from seqreads and stores every sequence into a list of strings called seqreads_str

print(f'Found {len(seqreads)} sequencing reads in barcpdes_R1 file') #prints total number of entries in seqreads
print (f'Found {len(seqreads_str)} sequences in seqreads') #prints total number of sequences in seqreads_str

Found 10000 sequencing reads in barcpdes_R1 file
Found 10000 sequences in seqreads


In [2]:
#defining reverse_complement function to get the reverse complement, append to the revcomplement list and join the nucleotides

def reverse_complement(seq): #"seq" is what the user will input, which will be fed through the reverse_complement function
    """Get reverse complement of a DNA sequence.
    
    Parameters
    -----------
    seq : str
        Uppercase DNA sequence.
        
    Returns
    -------
    str
        Reverse complement of the sequence in upper case.
        
    Example
    --------
    >>> reverse_complement('ATGCAC')
    'GTGCAT'
    
    """
    rc_dict = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G', 'N': 'N'} #rc_dict is the dictionary that will be iterated through to get the reverse_complement of each sequence
    revcomplement = [] #revcomplement = [] is the empty list that each reverse complement sequence will be added to
    for nt in reversed(seq.upper()): #seq.upper makes all the nucleotides in each sequence into an uppercase letter
        revcomplement.append(rc_dict[nt]) #adds each reverse complemented sequence to the revcomplement list
    return ''.join(revcomplement) #joins each nucleotide in each specific sequence together


#####################################################################################################################



#defining HA_barcode function to find HA specific pattern (through regex) and search through seqread
def HA_barcode(seqread, bclen=16, upstream='AGGCGGCCGC', HA='CCGGATTTGCATATAATGATGCACCAT'):
    """Identify barcode with known upstream sequence.
    
    Parameters
    ----------
    seqread : str
        Nucleotide sequence matching UPSTREAM-BARCODE read in reverse orientation.
    bclen : int
        Length of barcode
    upstream: str
        Sequence upstream of the barcode.
    HA : str
        Distinguishing HA genes within seqreads_string
        
    Returns
    -------
    str or None
        Sequence of the barcode in the forward orientation, or `None` if no match to expected barcoded sequence.
        

        
    """
    HA_matcher = re.compile(f'^(?P<barcode>[ACGT]{{{bclen}}})' + reverse_complement(upstream) + reverse_complement(HA)) #finding HA pattern through regex
    HA_search = HA_matcher.search(seqread) #searching seqread using the regex pattern stated above
    if not HA_search: #if it's not HA pattern, skip it
        return None
    else:
        return reverse_complement(seqread[: bclen]) #else, print the barcode


######################################################################################################################


# defining NA_barcode function to find NA specific pattern (through regex) and search through seqread
def NA_barcode(seqread, bclen=16, upstream='AGGCGGCCGC', NA='CACGATAGATAAATAATAGTGCACCAT'):
    """Identify barcode with known upstream sequence.
    
    Parameters
    ----------
    seqread : str
        Nucleotide sequence matching UPSTREAM-BARCODE read in reverse orientation.
    bclen : int
        Length of barcode
    upstream: str
        Sequence upstream of the barcode.
    HA : str
        Distinguishing HA genes within seqreads_string
        
    Returns
    -------
    str or None
        Sequence of the barcode in the forward orientation, or `None` if no match to expected barcoded sequence.
        
        
    """
    NA_matcher = re.compile(f'^(?P<barcode>[ACGT]{{{bclen}}})' + reverse_complement(upstream) + reverse_complement(NA))
    NA_search = NA_matcher.search(seqread)
    if not NA_search:
        return None
    else:
        return reverse_complement(seqread[: bclen])


In [7]:
#got help from Tim Yu
HA_dict = {}
NA_dict = {}


HA_count = 0
NA_count = 0

for seq in seqreads_str:
    HA = HA_barcode(seq)
    if HA is not None:
        if HA not in HA_dict:
            HA_dict[HA] = 1 #add the barcode as the key and give it a value of one
        else:
            HA_dict[HA] += 1 #updates the value of each key
    NA = NA_barcode (seq)
    if NA is not None:
        if NA not in NA_dict:
            NA_dict[NA] = 1 #add the barcode as the key and give it a value of one
        else:
            NA_dict[NA] += 1 #updates the value of each key

            

HA_dict
print (f'the sum of HA is {sum(HA_dict.values())}')
NA_dict
print (f'the sum of NA is {sum(NA_dict.values())}')

the sum of HA is 5245
the sum of NA is 3907


2. What is the HA barcode with the most counts (and how many counts)? Also answer the same question for NA.

In [8]:
all_HA_values = HA_dict.values()
max_HA_value = max(all_HA_values)

all_HA_keys = HA_dict.keys()
max_HA_keys = max(HA_dict, key=lambda key: HA_dict[key])

print(f'The maximum HA value is {max_HA_value} and the barcode is {max_HA_keys}')


all_NA_values = NA_dict.values()
max_NA_value = max(all_NA_values)

all_NA_keys = NA_dict.keys()
max_NA_keys = max(NA_dict, key=lambda key: NA_dict[key])

print (f'The maximum NA value is {max_NA_value} and the barcode is {max_NA_keys}')


The maximum HA value is 155 and the barcode is CCCGACCCGACATTAA
The maximum NA value is 152 and the barcode is ACCAGTTCTCCCCGGG
